In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
from collections import deque

def calculate_mar(face_landmarks):
    p_vertical1 = face_landmarks.landmark[13]
    p_vertical2 = face_landmarks.landmark[14]
    p_horizontal1 = face_landmarks.landmark[78]
    p_horizontal2 = face_landmarks.landmark[308]

    vertical_dist = np.linalg.norm([p_vertical1.x - p_vertical2.x, p_vertical1.y - p_vertical2.y])
    horizontal_dist = np.linalg.norm([p_horizontal1.x - p_horizontal2.x, p_horizontal1.y - p_horizontal2.y])
    
    if horizontal_dist > 0:
        return vertical_dist / horizontal_dist
    return 0.0


try:
    svm_model = joblib.load('svm_yawn_detector.joblib')
    scaler = joblib.load('scaler.joblib')
except FileNotFoundError:
    print("Error: File model atau scaler tidak ditemukan.")
    exit()


mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)


WINDOW_SIZE = 20
mar_buffer = deque(maxlen=WINDOW_SIZE)
prediction_text = "NORMAL"


MOUTH_INDICES = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291] 

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Tidak bisa membuka kamera.")
    exit()


while True:
    success, frame = cap.read()
    if not success:
        break
    h, w, _ = frame.shape

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    
    current_mar = 0.0
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            current_mar = calculate_mar(face_landmarks)
            
            
            # 1. Bounding Box Wajah (dari semua landmark)
            all_x = [lm.x * w for lm in face_landmarks.landmark]
            all_y = [lm.y * h for lm in face_landmarks.landmark]
            face_x1, face_y1 = int(min(all_x)), int(min(all_y))
            face_x2, face_y2 = int(max(all_x)), int(max(all_y))

            # 2. Bounding Box Mulut (hanya dari landmark mulut)
            mouth_x = [face_landmarks.landmark[i].x * w for i in MOUTH_INDICES]
            mouth_y = [face_landmarks.landmark[i].y * h for i in MOUTH_INDICES]
            mouth_x1, mouth_y1 = int(min(mouth_x)), int(min(mouth_y))
            mouth_x2, mouth_y2 = int(max(mouth_x)), int(max(mouth_y))
            
            # Gambar kotak hijau di sekeliling wajah
            cv2.rectangle(frame, (face_x1, face_y1), (face_x2, face_y2), (0, 255, 0), 2)
            
            # Gambar kotak biru di sekeliling mulut
            cv2.rectangle(frame, (mouth_x1, mouth_y1), (mouth_x2, mouth_y2), (255, 0, 0), 1)

    mar_buffer.append(current_mar)

    if len(mar_buffer) == WINDOW_SIZE:
        features = np.array([[np.mean(mar_buffer), np.max(mar_buffer), np.std(mar_buffer)]])
        scaled_features = scaler.transform(features)
        prediction = svm_model.predict(scaled_features)[0]
        
        if prediction == 1:
            prediction_text = "MENGUAP"
        else:
            prediction_text = "NORMAL"


    cv2.putText(frame, f"Status: {prediction_text}", 
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if prediction_text == "NORMAL" else (0, 0, 255), 2)
    cv2.putText(frame, f"MAR: {current_mar:.2f}", 
                (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)


    cv2.imshow('Deteksi Menguap Real-time', frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\kenny\anaconda3\envs\DROWSINESS_DETECTOR\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not ha